In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import scipy.io
import time
drive_path = '/Volumes/Brain2016'
manifest_path = os.path.join(drive_path,'BrainObservatory/manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)
stimulustype='spontaneous'
good=np.load('good_dfof3_cell_ids.npy')

In [3]:
L23_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'], imaging_depths=[175], cre_lines=['Cux2-CreERT2'],stimuli=[stimulustype])
L4_Cux2_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'], imaging_depths=[275], cre_lines=['Cux2-CreERT2'],stimuli=[stimulustype])
L4_Rorb_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'],  cre_lines=['Rorb-IRES2-Cre'],stimuli=[stimulustype])
L4_Scnn1a_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'],  cre_lines=['Scnn1a-Tg3-Cre'],stimuli=[stimulustype])

celltype='L4_Scnn1a_VISp'

In [11]:
## get one experiment at a time to pick data
i=0
for experiment in eval(celltype):
    data=boc.get_ophys_experiment_data(ophys_experiment_id = experiment['id'])
#select good cells with dfof >3%
    cells=data.get_cell_specimen_ids()
    good_cells=[n for n in cells if n in good]  
    timestamps,exp_1_dff=data.get_dff_traces(cell_specimen_ids=good_cells)
    running_speed,_ = data.get_running_speed()
    stim_time=data.get_spontaneous_activity_stimulus_table() 
    for section in range(len(stim_time)):
        starting=stim_time['start'][section]
        ending=stim_time['end'][section]
        timestamps=timestamps[starting:ending]
        exp_1=exp_1_dff[:,starting:ending]          
        running_speed=running_speed[starting:ending]
        i=i+1
        matlabdict[cell_line+str(i)]=exp_1

0 30696 39576
1 66794 75704
2 66782 75689
3 22569 31451
4 73142 82022
5 22608 31508
6 73288 82186
7 30688 39567
8 30701 39580
9 22567 31446
10 73133 82011
11 22573 31453
12 73147 82028
13 66804 75715
14 30708 39588
15 66791 75700


In [ ]:
savingfile='/Volumes/Brain2016/modelling/'+ stimulustype+ '/'+cell_line
print savingfile
scipy.io.savemat(savingfile,matlabdict)